
# 5. RAG(검색증강생성) 고급

## 실습하기



In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-test"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [3]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━

In [4]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

371


In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
아래 문맥을 고려하여 질문에 답해 주세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChain의 개요에 대해 ")

'LangChain은 대형 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 라이프사이클의 모든 단계를 간소화하는 데 중점을 두고 있습니다. 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 빌딩 블록, 구성 요소 및 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축하고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 체인을 검사, 모니터링 및 평가함으로써 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 여러 오픈 소스 라이브러리로 구성되어 있으며, 각 라이브러리는 특정 기능을 제공합니다. 예를 들어, `langchain-core`는 기본 추상화 및 LangChain 표현 언어를 제공하고, `langchain-openai`와 같은 통합 패키지는 특정 모델과의 통합을 지원합니다. LangGraph는 LLM을 사용하여 강력하고 상태 유지가 가능한 다중 액터 애플리케이션을 구축할 수 있도록 도와줍니다.\n\n또한, LangChain은 표준화된 구성 요소 인터페이스, 오케스트레이션, 관찰 가능성 및 평가 기능을 제공하여 개발자가 복잡한 애플리케이션을 쉽게 구축하고 관리할 수 있도록 지원합니다.'

## ⑶ 검색쿼리 고안

### HyDE (Hypothetical Document Embeddings)

In [7]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
다음 질문에 대한 답을 한 문장으로 작성해 주세요.

질문: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [8]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChain의 개요에 대해")

'LangChain은 개발자들이 AI 애플리케이션을 쉽게 구축할 수 있도록 돕기 위해 설계된 Python 패키지이자 회사입니다. LangChain의 목표는 다양한 AI 모델과 관련 구성 요소를 통합하여 표준화된 인터페이스를 제공하고, 복잡한 애플리케이션을 효율적으로 조정할 수 있는 기능을 제공하는 것입니다.\n\n### 주요 특징\n1. **표준화된 구성 요소 인터페이스**: 다양한 AI 모델과 구성 요소의 API가 서로 다르기 때문에, LangChain은 이러한 구성 요소에 대한 공통 인터페이스를 제공하여 개발자가 쉽게 전환하고 결합할 수 있도록 합니다.\n\n2. **오케스트레이션**: 복잡한 애플리케이션을 구축하기 위해 여러 구성 요소와 모델을 결합할 수 있는 기능을 제공합니다. 이를 통해 복잡한 제어 흐름을 지원하고, 인간의 개입이 필요한 경우에도 유연하게 대응할 수 있습니다.\n\n3. **가시성 및 평가**: 애플리케이션의 복잡성이 증가함에 따라, 내부에서 무슨 일이 일어나고 있는지를 이해하기 어려워집니다. LangChain은 개발자가 애플리케이션을 모니터링하고, 성능을 평가할 수 있는 도구를 제공합니다.\n\n### 구성 요소\n- **LLM(대형 언어 모델)**: 다양한 오픈 소스 LLM을 지원하며, 사용자가 자신의 모델을 쉽게 배포하고 사용할 수 있도록 돕습니다.\n- **도구 호출 및 구조화된 출력**: LangChain은 도구 호출 및 구조화된 출력을 지원하는 공통 방법을 제공합니다.\n- **LangGraph**: 복잡한 애플리케이션을 오케스트레이션하기 위한 라이브러리로, 노드와 엣지로 애플리케이션의 흐름을 표현할 수 있습니다.\n\nLangChain은 이러한 기능을 통해 개발자들이 AI 애플리케이션을 보다 쉽게 구축하고 관리할 수 있도록 지원합니다.'

### 다중 검색 쿼리 생성

In [9]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="검색어 목록")


query_generation_prompt = ChatPromptTemplate.from_template("""\
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하고,
세 가지 다른 검색 쿼리를 생성합니다.
거리 기반 유사도 검색의 한계를 극복하기 위해,
사용자의 질문에 대한 다양한 관점을 제공하는 것이 목표입니다.

질문: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [10]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChain의 개요에 대해 ")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 모든 단계, 즉 개발, 생산화, 배포를 간소화하는 데 중점을 두고 있습니다.\n\n### 주요 구성 요소:\n1. **개발**: LangChain의 오픈 소스 빌딩 블록, 구성 요소 및 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축하고, 스트리밍 및 인간-루프 지원을 제공합니다.\n   \n2. **생산화**: LangSmith를 사용하여 체인을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\n### 주요 라이브러리:\n- **`langchain-core`**: 기본 추상화 및 LangChain 표현 언어(LCEL).\n- **통합 패키지**: `langchain-openai`, `langchain-anthropic` 등과 같은 경량 패키지로, LangChain 팀과 통합 개발자가 공동 유지 관리합니다.\n- **`langchain`**: 체인, 에이전트 및 검색 전략으로 구성된 애플리케이션의 인지 아키텍처를 형성합니다.\n- **`langchain-community`**: 커뮤니티에서 유지 관리하는 제3자 통합.\n- **LangGraph**: LLM을 사용하여 강력하고 상태 유지가 가능한 다중 액터 애플리케이션을 구축할 수 있도록 지원합니다.\n- **LangServe**: LangChain 체인을 REST API로 배포합니다.\n- **LangSmith**: LLM 애플리케이션을 디버깅, 테스트, 평가 및 모니터링할 수 있는 개발자 플랫폼입니다.\n\nLangChain은 다양한 구성 요소를 조합하여 복잡한 애플리케이션을 구축할 수 있도록 지원하며, 표준화된 인터페이스를 제공하여 개발

## ⑷ 검색후 방법
### RAG-Fusion

In [11]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 각 문서의 내용(문자열)과 그 점수에 대응하는 사전을 준비함
    content_score_mapping = {}

    # 검색쿼리마다 반복
    for docs in retriever_outputs:
        # 검색결과 문서마다 반복
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 처음 나온 콘텐츠의 경우 점수가 0으로 초기화됨
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (순위 + k)) 점수를 합산함
            content_score_mapping[content] += 1 / (rank + k)

    # 점수가 높은 순으로 정렬
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [12]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChain의 개요에 대하여 ")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 라이프사이클의 모든 단계를 간소화합니다. 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 빌딩 블록, 구성 요소 및 타사 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축하고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 체인을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 여러 오픈 소스 라이브러리로 구성되어 있으며, 주요 구성 요소로는 `langchain-core`, 다양한 통합 패키지, `langchain`, `langchain-community`, LangGraph, LangServe, LangSmith 등이 있습니다. 이 프레임워크는 Python과 JavaScript에서 사용할 수 있으며, 다양한 튜토리얼과 가이드를 통해 사용자가 쉽게 시작할 수 있도록 지원합니다.'

### Cohere의 리랭크 모델 사용 준비

In [13]:
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

In [14]:
!pip install langchain-cohere==0.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0


### Cohere의 리랭크 모델 추가

In [15]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document


def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("LangChain의 개요에 대하여 ")

'LangChain은 대형 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 라이프사이클의 모든 단계를 간소화합니다. 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 빌딩 블록, 구성 요소 및 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축하고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 체인을 검사, 모니터링 및 평가함으로써 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 여러 오픈 소스 라이브러리로 구성되어 있으며, 주요 구성 요소로는 `langchain-core`, 다양한 통합 패키지, `langchain`, `langchain-community`, LangGraph, LangServe, LangSmith 등이 있습니다. 이 프레임워크는 Python과 JavaScript 라이브러리로 제공되며, 다양한 튜토리얼과 가이드를 통해 사용자가 쉽게 시작할 수 있도록 지원합니다. \n\nLangChain은 또한 표준화된 구성 요소 인터페이스, 복잡한 애플리케이션을 위한 오케스트레이션, 그리고 애플리케이션의 가시성과 평가를 지원하여 개발자가 더 효율적으로 작업할 수 있도록 돕습니다.'

## ⑸ 여러 개의 Retriever를 사용하기

###  LLM에 의한 라우팅

In [16]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [17]:
from enum import Enum

class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"

class RouteOutput(BaseModel):
    route: Route

route_prompt = ChatPromptTemplate.from_template("""\
질문에 대한 답변을 위해 적절한 Retriever를 선택하세요.

질문: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [18]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown retriever: {retriever}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [19]:
route_rag_chain.invoke("LangChain의 개요에 대하여 ")

'LangChain은 대형 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 라이프사이클의 모든 단계를 간소화하는 데 중점을 두고 있습니다. LangChain의 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 빌딩 블록, 구성 요소 및 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축하고, 스트리밍 및 인간-루프 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 체인을 검사, 모니터링 및 평가함으로써 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 여러 오픈 소스 라이브러리로 구성되어 있으며, 주요 구성 요소로는 `langchain-core`, 다양한 통합 패키지, `langchain`, `langchain-community`, LangGraph, LangServe, LangSmith 등이 있습니다. 이들 구성 요소는 독립적으로 사용 가능하며, 개발자는 자신의 사용 사례에 맞는 구성 요소를 선택하여 사용할 수 있습니다.\n\nLangChain은 또한 표준화된 구성 요소 인터페이스, 오케스트레이션, 관찰 가능성 및 평가 기능을 제공하여 복잡한 AI 애플리케이션을 쉽게 구축하고 관리할 수 있도록 돕습니다.'

In [20]:
route_rag_chain.invoke("서울의 오늘 날씨는?")

'서울특별시의 오늘 날씨에 대한 정보는 다음과 같습니다:\n\n- **최고/최저 기온**: 구체적인 수치는 문맥에 포함되어 있지 않지만, 오늘의 기온 변화를 확인할 수 있습니다.\n- **강수 확률**: 오늘의 강수 확률에 대한 정보는 제공되지 않았습니다.\n- **일출/일몰 시간**: 구체적인 시간은 언급되지 않았습니다.\n- **온도 기록**: 오늘의 온도 기록에 대한 정보도 포함되어 있지 않습니다.\n\n더 자세한 날씨 정보는 [AccuWeather](https://www.accuweather.com/ko/kr/seoul/226081/weather-today/226081) 또는 [The Weather Channel](https://weather.com/ko-KR/weather/today/l/82e46175f97c224acf6b95afc4934fbae0e4ba123adcee8a52b7be97c303467b) 웹사이트를 통해 확인하실 수 있습니다.'

### 하이브리드 검색 구현

In [21]:
!pip install rank-bm25==0.2.2

In [22]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [23]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [24]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChain의 개요에 대하여 ")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 라이프사이클의 모든 단계를 간소화합니다. 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 빌딩 블록, 구성 요소 및 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축하고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 체인을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 여러 오픈 소스 라이브러리로 구성되어 있으며, 주요 구성 요소로는 `langchain-core`, 다양한 통합 패키지, `langchain`, `langchain-community`, LangGraph, LangServe, LangSmith 등이 있습니다. 이 프레임워크는 Python과 JavaScript에서 사용할 수 있으며, 다양한 튜토리얼과 가이드를 통해 사용자가 쉽게 시작할 수 있도록 지원합니다. \n\nLangChain의 목표는 개발자가 LLM을 활용하여 애플리케이션을 쉽게 구축할 수 있도록 하는 것입니다.'